# Script to validate MCAC after major modifications

## Table of Contents
1. [Introduction](#Introduction)
2. [Open the files for each case](#Open-the-files-for-each-case)
    1. [Open the new case](#Open-the-new-case)
    2. [Open the reference case](#Open-the-reference-case)
3. [Comparison of both cases](#Comparison-of-both-cases)
    1. [Diffusion coefficient D](#Diffusion-coefficient-D)
    2. [Nc, L_box and N](#Nc,-L_box-and-N)
    3. [Flow regime](#Flow-regime)
    4. [Morphology](#Morphology)
    5. [Aggregate polydispersity](#Aggregate-polydispersity)
    6. [Total energy](#Total-energy)

## Introduction

This Python script aims at validating MCAC by comparing with a 'well known' case already simulated in a previous version of the code (published results). This is referred as "ref_case" compared with the new simulation "new_case".

In this particular code only coagulation takes place without other mechanisms of particle growth or nucleation. The aspects of the code to be validated are: the mobility (diffusion coefficient D), kinetics of aggregation (number of clusters: Nc, the size of the box: L_box, and the particle number concentration: N), flow regime (Gas Knudsen number) morpholoogy (Rg vs. Np, fractal dimension and prefactor), and aggregate polydispersity (GSD of Dv). Finally, the conservation of energy (only kinetic since interaction potential are not simulated) is tested.

In [ ]:
from pathlib import Path
from time import time

import matplotlib.pyplot as plt
import numpy as np
import xarray as xr

from scipy.optimize import root_scalar, root
from sklearn.linear_model import LinearRegression

from pymcac import MCAC
from pymcac import groupby_apply, groupby_agg
from pymcac import progress_compute, DaskDistribute

In [ ]:
def Dm_equation(Dmm, f_agg):
    Kn = 2 * lambda_g / Dmm
    
    cunningham = 1 + A1 * Kn + A2 * Kn * np.exp(-A3 / Kn)
    cunningham_prime = A1 * Kn + A2 * (A3 + Kn) * np.exp(-A3 / Kn)
    cunningham_prime2 = 2 * A1 * Kn + A2 * (A3 ** 2 / Kn + 2 * A3 + 2 * Kn) * np.exp(-A3 / Kn)
    
    return (
        3 * np.pi * mu_g * Dmm - f_agg * cunningham,
        3 * np.pi * mu_g + f_agg * cunningham_prime / Dmm,
        - f_agg * cunningham_prime2 / Dmm ** 2
    )

dmm_norm = 1e-8
def normalized_Dm_equation(Dmm, f_agg):
    f_norm = 1 / f_agg / 1000
    
    f, df, ddf = Dm_equation(Dmm * dmm_norm, f_agg)
    
    return (
        f/f_norm,
        df / f_norm * dmm_norm,
        ddf / f_norm * dmm_norm * dmm_norm
    )

def compute_Dm(f_agg, x0=1e-8):
    x0 = x0 / dmm_norm
    rootresults = root_scalar(normalized_Dm_equation, args=(f_agg,),
                              fprime=True,  fprime2=True,
                              x0=x0, x1=10*x0)
    if not rootresults.converged:
#         raise ValueError("Warning, compute_Dm failed to find a solution ")
        print("Warning, compute_Dm failed to find a solution ")
        return np.nan
    return rootresults.root * dmm_norm


def vectorized_compute_Dm(vect_f_agg, x0=1e-8):
    res = np.empty_like(vect_f_agg)
    for i, f_agg in enumerate(vect_f_agg):
        res[i] = x0 = compute_Dm(f_agg, x0=x0)
    return res

In [ ]:
def add_missing_quantities(xAggregates, xSpheres):
    
    xAggregates["log_Rv"] = np.log((6 * xAggregates.Volume / np.pi)) / 3
    xAggregates["Ec"] = 0.5 * xAggregates.Volume * rhop_0 * (xAggregates.lpm / xAggregates.Deltat) ** 2
    
    Dm = xr.apply_ufunc(vectorized_compute_Dm,
                        xAggregates.f_agg,
                        dask="parallelized", output_dtypes=[float])
    xAggregates["Kn"] = 2 * lambda_g / Dm

    xMeanRp = groupby_agg(xSpheres,
                          by=["Time", "Label"],
                          lengths=(xAggregates.k.size,),
                          sort=True,
                          agg=[("Radius", "mean", "Radius")])
    xMeanRp = xMeanRp.chunk(xAggregates.chunks)
    
    xAggregates["DgOverDp"] = xAggregates.Rg / xMeanRp.data
    
    return xAggregates, xSpheres

In [ ]:
def time_averaged_quantities(xAggregates, xSpheres, times):
    
    time_averaged = groupby_agg(xAggregates,
                                by="Time",
                                set_index=False,
                                lengths=(times.size,),
                                sort=True,
                                agg=[
        ("f_agg", "mean","f_agg"),
        ("Np", "mean","Np"),
        ("Rg", "mean","Rg"),
        ("DgOverDp", "mean","DgOverDp"),
        ("Kn", "mean","Kn"),
        ("BoxSize", "first","BoxSize"),
        ("Nc", "size","Np"),
        ("Ec", "sum","Ec"),
        ("fv", "sum","Volume"),
        ("Rv_geo", "mean","log_Rv"),
        ("sv_geo", "std","log_Rv")
    ])
    BoxVolume = time_averaged.BoxSize ** 3
    time_averaged["fv"] /= BoxVolume
    time_averaged["Rv_geo"] = np.exp(time_averaged.Rv_geo)
    time_averaged["sv_geo"] = np.exp(time_averaged.sv_geo)
    
    time_averaged["Nc density"] = time_averaged.Nc / BoxVolume
    time_averaged["Diffusion"] = k_B * T_g / time_averaged.f_agg

    Dm = xr.apply_ufunc(vectorized_compute_Dm,
                        time_averaged.f_agg,
                        dask="parallelized", output_dtypes=[float])   
    time_averaged["Kn_avg"] = 2 * lambda_g / Dm
    
    xMeanRp = groupby_agg(xSpheres,
                          by="Time",
                          set_index=False,
                          lengths=(times.size,),
                          sort=True,
                          agg=[("Radius", "mean", "Radius")])
    xMeanRp = xMeanRp.chunk(time_averaged.chunks)
    time_averaged["DgOverDp_avg"] = time_averaged.Rg / xMeanRp.data
    
    return time_averaged

In [ ]:
def read(path, tmax=None, ncmax=None):
    # The folder with all .h5 and .xmf files
    MCACSimulation = MCAC(path)

    # Read all data
    xSpheres = MCACSimulation.xspheres
    xAggregates = MCACSimulation.xaggregates

    times = MCACSimulation.times
    
    print("filtering")
    print(" time")
    if tmax is None:
        tmax = MCACSimulation.times[-1]
    times = np.linspace(MCACSimulation.times[0], tmax, 100)
    ts = np.unique((MCACSimulation.times<times[:,np.newaxis]).argmin(axis=1))
    print("  xSpheres")
    xSpheres = xSpheres.where(xSpheres.Time.isin(MCACSimulation.times[ts]), drop=True)
    print("  xAggregates")
    xAggregates = xAggregates.where(xAggregates.Time.isin(MCACSimulation.times[ts]), drop=True)

#     if ncmax is not None:
#         print(" ncmax")
#         print("  xSpheres")
#         xSpheres = xSpheres.where(xSpheres.Label < ncmax, drop=True)
#         print("  xAggregates")
#         xAggregates = xAggregates.where(xAggregates.Label < ncmax, drop=True)
    print("done")
    
    # Per aggregate computations
    xAggregates, xSpheres = add_missing_quantities(xAggregates, xSpheres)
    
    # Per time-step computations
    time_averaged = time_averaged_quantities(xAggregates, xSpheres, times)

    return MCACSimulation, time_averaged

In [ ]:
def plot_compare(ds1, ds2, x, y, xlabel=None, ylabel=None, **kwargs):
    fig = plt.figure(figsize=(10, 6), dpi=80, facecolor='w', edgecolor='k')
    fig.set_tight_layout(False)
    
    kwargs["ax"] = kwargs.get("ax", plt.gca())

    if x is None:
        x = next(iter(ds1.coords))
        
    if isinstance(y, str):
        y = [y]
    
    if isinstance(ds1, xr.DataArray):
        ds1 = ds1.to_dataset()
    if isinstance(ds2, xr.DataArray):
        ds2 = ds2.to_dataset()

    df1 = ds1[[x]+y].to_dataframe()
    df2 = ds2[[x]+y].to_dataframe()
    
    if x in df1.columns:
        df1 = df1.set_index(x)
        df2 = df2.set_index(x)
    df1 = df1.sort_index()
    df2 = df2.sort_index()
    
    df1.plot(**kwargs, style="-")
    df2.plot(**kwargs, style="--")

    if xlabel is not None:
        plt.xlabel(xlabel)
    if ylabel is not None:
        plt.ylabel(ylabel)
        
    plt.legend([f"New {varname}" for varname in y] + [f"Ref {varname}" for varname in y])
    plt.show()

In [ ]:
# PARAMETERS (check this parameters before running the script!)
T_g = 1700  # temperature in K
P_g = 101300  # pressure in Pa
rhop_0 = 1800  # particle bulk density in kg/m^3
k_B = 1.38066E-23  # Boltzmann constant in J/K

# gas mean free path and viscosity
lambda_g = 66.5E-9 * (101300 / P_g) * (T_g / 293.15) * (1 + 110 / 293.15) / (1 + 110 / T_g)  # in m
mu_g = 18.203E-6 * (293.15 + 110) / (T_g + 110) * (T_g / 293.15) ** 1.5  # in Ps*s

# for Cunningham slip correction factor: Cc
A1 = 1.142
A2 = 0.558
A3 = 0.999

#reference_path = Path("/stockage/samba/Partages/public/MCAC_validation/02_VARYING_DP/02p3_Dp10nm_np0_3200/run1")
reference_path = Path("/Data/WORK/Projets/SRC/MCAC/validation/monodisperse_data_ref/")
result_path = Path("/Data/WORK/Projets/SRC/MCAC/validation/monodisperse_data")

In [ ]:
client = DaskDistribute().start()
client

In [ ]:
print("Reading new")
newSimulation, new_time_averaged = read(result_path, tmax=0.39e-3, ncmax=100)

print("Reading reference")
refSimulation, ref_time_averaged = read(reference_path, tmax=0.39e-3, ncmax=100)

print("Start compute")
start = time()
new_time_averaged, ref_time_averaged = progress_compute(new_time_averaged, ref_time_averaged)
new_time_averaged = new_time_averaged.swap_dims({"k": "Time"})
ref_time_averaged = ref_time_averaged.swap_dims({"k": "Time"})
print(f"Total Compute time : {time() - start} s")

In [ ]:
client.close()

In [ ]:
new_time_averaged

In [ ]:
print(f" Ref total residence time: "
      f"{float(ref_time_averaged.Time[-1] - ref_time_averaged.Time[0]) * 1e3} (ms) "
      f"({ref_time_averaged.Time.size} it)")
print(f" New total residence time: "
      f"{float(new_time_averaged.Time[-1] - new_time_averaged.Time[0]) * 1e3} (ms) "
      f"({new_time_averaged.Time.size} it)")

In [ ]:
plot_compare(new_time_averaged.fv * 1e+06, ref_time_averaged.fv * 1e+06,
             x=None, xlabel="Time (s)",
             y="fv", ylabel="volume fraction (ppm)",
             loglog=False)

In [ ]:
plot_compare(new_time_averaged, ref_time_averaged,
             x="Np", xlabel="Number of monomers per aggregate (mean)",
             y="Diffusion", ylabel="Diffusion coefficient (m^2/s)",
             loglog=True)

In [ ]:
plot_compare(new_time_averaged, ref_time_averaged,
             x=None, xlabel="Time (s)",
             y="Nc", ylabel="Number of aggregate (-)",
             loglog=True)
plot_compare(new_time_averaged, ref_time_averaged,
             x=None, xlabel="Time (s)",
             y="BoxSize", ylabel="Box size (m)",
             loglog=True)
plot_compare(new_time_averaged, ref_time_averaged,
             x=None, xlabel="Time (s)",
             y="Nc density", ylabel="Particle number concentration (-)",
             loglog=True)
print(f"Ref case initialized with {int(ref_time_averaged.Nc[0])} monomers")
print(f"New case initialized with {int(new_time_averaged.Nc[0])} monomers")

In [ ]:
plot_compare(new_time_averaged, ref_time_averaged,
             x=None, xlabel="Time (s)",
             y="Kn_avg", ylabel="Gas Knudsen number",
             loglog=True)
plot_compare(new_time_averaged, ref_time_averaged,
             x=None, xlabel="Time (s)",
             y="Kn", ylabel="Gas Knudsen number",
             loglog=True)
plot_compare(new_time_averaged, ref_time_averaged,
             x=None, xlabel="Time (s)",
             y=["Kn", "Kn_avg"], ylabel="Gas Knudsen number (computed of averages)",
             loglog=True)

In [ ]:
plot_compare(new_time_averaged, ref_time_averaged,
             x="Np", xlabel="Number of monomers per aggregate (mean)",
             y="Rg", ylabel="Radius of gyration (m)",
             loglog=True)

In [ ]:
plot_compare(new_time_averaged, ref_time_averaged,
             x="Np", xlabel="Number of monomers per aggregate (mean)",
             y="DgOverDp_avg", ylabel="DgOverDp",
             loglog=True)
plot_compare(new_time_averaged, ref_time_averaged,
             x="Np", xlabel="Number of monomers per aggregate (mean)",
             y="DgOverDp", ylabel="DgOverDp",
             loglog=True)
plot_compare(new_time_averaged, ref_time_averaged,
             x="Np", xlabel="Number of monomers per aggregate (mean)",
             y=["DgOverDp", "DgOverDp_avg"], ylabel="DgOverDp",
             loglog=True)

In [ ]:
# The population-based fractal dimension and prefactor
model = LinearRegression()
model.fit(np.log(ref_time_averaged.DgOverDp_avg).values[:, np.newaxis],
          np.log(ref_time_averaged.Np))
print(f"Ref fractal Law: {np.exp(model.intercept_)} x^{model.coef_[0]}")

# model = LinearRegression()
model.fit(np.log(new_time_averaged.DgOverDp_avg).values[:, np.newaxis],
          np.log(new_time_averaged.Np))
print(f"New fractal Law: {np.exp(model.intercept_)} x^{model.coef_[0]}")

In [ ]:
plot_compare(new_time_averaged, ref_time_averaged,
             x="Np", xlabel="Number of monomers per aggregate (mean)",
             y="Rv_geo", ylabel="Geometric mean vol-eq. radius, R_v (nm)",
             loglog=False)
plot_compare(new_time_averaged, ref_time_averaged,
             x="Np", xlabel="Number of monomers per aggregate (mean)",
             y="sv_geo", ylabel="Geometric standard deviation, $\sigma_{g,rv}$ (-)",
             loglog=False)

In [ ]:
new_time_averaged["Kinetic energy per aggregate"] = new_time_averaged.Ec / new_time_averaged.Nc
ref_time_averaged["Kinetic energy per aggregate"] = ref_time_averaged.Ec / ref_time_averaged.Nc
plot_compare(new_time_averaged, ref_time_averaged,
             x="Np", xlabel="Number of monomers per aggregate (mean)",
             y="Kinetic energy per aggregate", ylabel="Kinetic energy per aggregate",
             loglog=False)